In [1]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
df = pd.read_json(open("data.json", "r", encoding="utf8"))

In [3]:
df

,title,url,author,year,text,themes
0,"[28, 508, 152, 720]",https://poets.org/poem/body-and-soul-ii,602,2002,"[2221, 1012, 2, 2221, 273, 902, 128, 472, 57, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,[2668],https://poets.org/poem/novel,311,2002,"[286, 3904, 1, 2778, 249, 19, 1268, 643, 410, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,[178],https://poets.org/poem/flying,3478,2002,"[12, 602, 11, 1574, 197, 113, 402, 13, 2428, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[1113, 2862, 88, 289, 1171, 2335]",https://poets.org/poem/photograph-people-danci...,2361,2002,"[267, 16, 35, 5, 5, 27, 1229, 1823, 3196, 48, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[246, 1113]",https://poets.org/poem/war-photograph,2125,2002,"[596, 50, 124, 467, 163, 612, 133, 77, 29, 44,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...
17069,"[66, 148, 466, 506, 1171, 512, 1513]",https://www.poetryfoundation.org/poetrymagazin...,1826,1990,"[148, 35, 466, 4186, 26, 43, 3285, 278, 241, 5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
17070,"[66, 88]",https://www.poetryfoundation.org/poetrymagazin...,2883,1990,"[88, 16, 35, 90, 506, 733, 68, 959, 397, 253, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
17071,"[66, 180, 8, 9]",https://www.poetryfoundation.org/poetrymagazin...,5,2005,"[9, 24, 22, 299, 140, 467, 25, 299, 995, 81, 7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
17072,"[3966, 549]",https://www.poetryfoundation.org/poetrymagazin...,2043,1990,"[465, 569, 218, 465, 381, 190, 54, 549]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [4]:
# one hot encoding for authors
def encode_authors(author_code):
    qty = df.author.max()
    result = [0] * (qty + 1)
    result[author_code] = 1
    return result

df.author = df.author.apply(encode_authors)

In [5]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [6]:
df['joined_text'] = df['text'] + df['title']
df['X2'] = df['themes'] + df['author'] 
train_df, test_df = model_selection.train_test_split(df, test_size=0.1, random_state=42)
X1_train = keras.preprocessing.sequence.pad_sequences(train_df['joined_text'].to_list(), maxlen=max_len, padding='post')
X2_train = np.stack(train_df['X2'])
Y_train = np.stack(train_df['year'])
X1_test = keras.preprocessing.sequence.pad_sequences(list(test_df['joined_text']), maxlen=max_len, padding='post')
X2_test = np.stack(test_df['X2'])
Y_test = np.stack(test_df['year'])

### Model 

In [7]:
embedding_dim =64
model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(64,activation='relu'),
  keras.layers.Dense(32,activation='relu'),
  keras.layers.Dense(16,activation='relu'),
  keras.layers.Dense(8,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model1.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                614464    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9

In [8]:
epochs = 10
checkpoint = keras.callbacks.ModelCheckpoint('year_prediction_model.h5', 
    monitor='val_MAE',
    mode='min',
    save_best_only=True)


model1.fit(X1_train, Y_train,
          batch_size=128,
          validation_data=(X1_test,Y_test),
          epochs=epochs,
          callbacks=[checkpoint])

Epoch 1/10
121/121 [==============================] - 2s 19ms/step - loss: 1467502.7500 - MAE: 887.5441 - val_loss: 12857.0469 - val_MAE: 90.0296
Epoch 2/10
121/121 [==============================] - 2s 17ms/step - loss: 6418.8701 - MAE: 55.5822 - val_loss: 4635.1035 - val_MAE: 47.0525
Epoch 3/10
121/121 [==============================] - 2s 17ms/step - loss: 4091.7778 - MAE: 42.2608 - val_loss: 4230.7871 - val_MAE: 48.2678
Epoch 4/10
121/121 [==============================] - 2s 17ms/step - loss: 3386.5022 - MAE: 37.4277 - val_loss: 3503.3662 - val_MAE: 38.0188
Epoch 5/10
121/121 [==============================] - 2s 18ms/step - loss: 2940.5154 - MAE: 33.7252 - val_loss: 3208.2295 - val_MAE: 35.3210
Epoch 6/10
121/121 [==============================] - 2s 17ms/step - loss: 2660.6121 - MAE: 31.2198 - val_loss: 2944.3259 - val_MAE: 33.5915
Epoch 7/10
121/121 [==============================] - 2s 17ms/step - loss: 2467.3010 - MAE: 29.5165 - val_loss: 2923.0190 - val_MAE: 32.8291
Epoch 8/

In [9]:
score1 = model1.evaluate(np.array(X1_test), np.array(Y_test)) 

print("Test Score:", score1[0])
print("Test Accuracy:", score1[1])

54/54 [==============================] - 0s 2ms/step - loss: 3111.9163 - MAE: 40.6188
Test Score: 3111.916259765625
Test Accuracy: 40.61882400512695


In [10]:
gf

NameError: name 'gf' is not defined

In [ ]:
model2 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=2,
                           output_dim=8,
                           input_length=len(X2_train[0])),
  keras.layers.Flatten(),
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model2.build()

In [ ]:
model2.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

In [ ]:
model2.summary()

In [ ]:
model2.fit(X2_train, Y_train, epochs=5, validation_split=0.1, batch_size=128,)

In [ ]:
score2 = model2.evaluate(np.array(X2_test), np.array(Y_test)) 

print("Test Score:", score2[0])
print("Test Accuracy:", score2[1])

In [ ]:
y_pred1 = model1.predict(X1_test)
y_pred2 = model2.predict(X2_test)
result = (y_pred1 + y_pred2)/2

In [ ]:
keras.metrics.mean_absolute_error(
    y_pred1.flatten(), y_pred2.flatten()
)

### Multi-input model

In [ ]:
text_input = keras.Input(shape=(max_len,))
categorical_input = keras.Input(shape=(len(X2_train[0]),))

text_embedding = layers.Embedding(max_features, 64)(text_input)
categorical_embedding = layers.Embedding(2, 8)(categorical_input)

flat_text = layers.Flatten()(text_embedding)
flat_categories = layers.Flatten()(categorical_embedding)

concatenated = keras.layers.Concatenate()([flat_text, flat_categories])

dense1 = keras.layers.Dense(2000, activation='relu', )(concatenated)
dense2 = keras.layers.Dense(500, activation='relu', )(dense1)
dense3 = keras.layers.Dense(100, activation='relu', )(dense2)
out = keras.layers.Dense(1, activation='relu', )(dense3)


united_model = keras.Model(inputs=[text_input, categorical_input], outputs=out)


In [ ]:
united_model.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])
united_model.summary()

In [ ]:
united_model.fit([X1_train, X2_train], Y_train, epochs=5, validation_split=0.1)

In [ ]:
score3 = united_model.evaluate([np.array(X1_test), np.array(X2_test)], np.array(Y_test)) 

print("Test Score:", score3[0])
print("Test Accuracy:", score3[1])

In [ ]:
output_dim = 100

inputs = keras.Input(shape=(None,))

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation='relu', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="mean_squared_error", optimizer="adam", metrics=["MAE"])


In [ ]:
model3.summary()

In [ ]:
batch_size = 64
epochs = 5
model3.fit(X1_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

In [ ]:
accr = model3.evaluate(X1_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))